In [5]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import scipy.stats as st
import matplotlib.pyplot as plt
import random

In [6]:
df = pd.read_csv("/home/aix7101/jeong/CeeBERT/Early_Exit_Confidence_data_yelp_max_exits(3,4,12)_difference.csv")
df.head(5)

,Unnamed: 0,Thi_layer_P,Fou_layer_P,Last_layer,PProb_thi,PProb_fou,PProb_las,True_labels
0,0,0,0,0,0.753920,0.800089,0.845397,0
1,1,0,1,1,0.594112,0.815985,0.813593,1
2,2,0,0,0,0.822457,0.724403,0.820971,0
3,3,0,0,0,0.743167,0.728612,0.674946,0
4,4,1,1,1,0.638572,0.865548,0.837171,1


In [7]:
action_set_1 = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0] # threshold candidates

In [8]:
# accuracy_imdb = []
# for j in df.columns:
#   accuracy = 0
#   for i in range(df.shape[0]):
#       if df[j][i] == df['True_labels'][i]:
#           accuracy += 1
#       else:
#           pass
#   print("Accuracy = ", accuracy/df.shape[0])
#   accuracy_imdb.append(accuracy)

In [9]:
print([action_set_1[0], action_set_1[1]])

[0.5, 0.6]


In [10]:
action_set = []
act_mat = np.zeros([len(action_set_1), len(action_set_1), 2])
for i in range(act_mat.shape[0]):
    for j in range(act_mat.shape[1]):
        # if i>j:     
        act_mat[i][j] = np.array([action_set_1[i], action_set_1[j]])
        action_set.append(act_mat[i][j])

In [11]:
len(action_set) # 2개의 threshold 후보를 택해서 사용, 6개의 후보 중 총 36개의 후보를 사용할 수 있음

36

In [12]:
print((action_set))

[array([0.5, 0.5]), array([0.5, 0.6]), array([0.5, 0.7]), array([0.5, 0.8]), array([0.5, 0.9]), array([0.5, 1. ]), array([0.6, 0.5]), array([0.6, 0.6]), array([0.6, 0.7]), array([0.6, 0.8]), array([0.6, 0.9]), array([0.6, 1. ]), array([0.7, 0.5]), array([0.7, 0.6]), array([0.7, 0.7]), array([0.7, 0.8]), array([0.7, 0.9]), array([0.7, 1. ]), array([0.8, 0.5]), array([0.8, 0.6]), array([0.8, 0.7]), array([0.8, 0.8]), array([0.8, 0.9]), array([0.8, 1. ]), array([0.9, 0.5]), array([0.9, 0.6]), array([0.9, 0.7]), array([0.9, 0.8]), array([0.9, 0.9]), array([0.9, 1. ]), array([1. , 0.5]), array([1. , 0.6]), array([1. , 0.7]), array([1. , 0.8]), array([1. , 0.9]), array([1., 1.])]


# One_UBERT_Cycle

In [13]:
def one_cycle(df, action_set, overhead):
    c_i_data_thi = []
    for i in df["PProb_thi"]:
        c_i_data_thi.append(i)

    c_i_data_six = []
    for i in df["PProb_six"]:
        c_i_data_six.append(i)

    c_l_data = []
    for i in df["PProb_las"]:
        c_l_data.append(i)
        
    # action_set_1 = [0.10*((i+10)/3) for i in range(20)]
    # action_set = []
    # for i in range(len(action_set_1)):
    #     if i%2 == 0:
    #         action_set.append(action_set_1[i])
    
    
    def reward(c_i_thi, c_i_six, c_l, action, overhead):
        if c_i_thi >= action_set[action][0]:
            reward = 0
        elif c_i_six >= action_set[action][1]:
            reward = c_i_six - c_i_thi - overhead[0] 
        else:
            reward = c_l - c_i_thi - overhead[0] - overhead[1]
        return reward
    
    
    def next_action(Q, c_tilde, N, t):
      # Q is the list of rewards
      # c_tilde is the hyperparameter
      # N is the number of times an action is chosen
        next_a = []
        for i in range(len(action_set)):

            next_a.append(Q[i] + c_tilde*((np.log(t)/N[i])**(0.5)))

        return next_a.index(max(next_a))
    
    
    def update_parameters(Q, N, t, c_i_data_thi, c_i_data_six, c_l_data, chosen_action, overhead):
        n = N[chosen_action]
        N[chosen_action] += 1
        Q[chosen_action] = (n*Q[chosen_action] + reward(c_i_data_thi[t], c_i_data_six[t], c_l_data[t], chosen_action, overhead))/N[chosen_action]
        return Q, N
    
    

    def initialize(Q, N, action_set, c_i_data_thi, c_i_data_six, c_l_data, overhead):
        for t in range(len(action_set)):
            update_parameters(Q, N, t, c_i_data_thi, c_i_data_six, c_l_data, t, overhead)
        # print(N)
        return Q, N
    
    
    
    
    def oracle_action(overhead):
        oracle_reward = []
        for i in action_set:
            o_reward = 0 
            for t in range(df.shape[0]):
                if c_i_data_thi[t] >= i[0]:
                    reward = 0
    #                 predictions.append(df["Fou_layer_P"][t])
                elif c_i_data_six[t] >= i[1]:
                    reward = c_i_data_six[t] - c_i_data_thi[t] - overhead[0] 
                else:
                    reward = c_l_data[t] - c_i_data_thi[t] - overhead[0] - overhead[1] 
                o_reward += reward
    #         print(oracle_reward)
            oracle_reward.append(o_reward/df.shape[0])

        return oracle_reward.index(max(oracle_reward)), oracle_reward
    oracle_action, o_reward = oracle_action(overhead)
    
    
    print("Oracle action =====> ", oracle_action)
    
    
    
    def one_UCB_cycle(Q, N, c_tilde, c_i_data_thi, c_i_data_six, c_l_data, overhead, df):
        action_chosen = []
        predictions = []
        rewards = []
        count_1 = 0
        count_2 = 0
        for t in range(df.shape[0]):
            if t < len(action_set):
                action = action_set[t]
                action_chosen.append(action)
                if c_i_data_thi[t] >= action_set[t][0]:
                    reward = 0
                    predictions.append(df["Thi_layer_P"][t])
                    rewards.append(reward)
                elif c_i_data_six[t] >= action_set[t][1]:
                    reward = c_i_data_six[t] - c_i_data_thi[t] - overhead[0]
                    predictions.append(df["Six_layer_P"][t])
                    rewards.append(reward)
                else:
                    reward = c_l_data[t] - c_i_data_thi[t] - overhead[0] - overhead[1]
                    predictions.append(df["Last_layer"][t])
                    rewards.append(reward)

            else:

                action = next_action(Q, c_tilde, N, t)
                # print(action)
                action_chosen.append(action)
                if c_i_data_thi[t] >= action_set[action][0]:
                    reward = 0
                    predictions.append(df["Thi_layer_P"][t])
                    rewards.append(reward)
                    count_1+=1
                    for i in range(len(action_set)):
                        if action_set[i][0] == action_set[action][0] and action_set[i][1] != action_set[action][1]:
                            Q, N = update_parameters(Q, N, t, c_i_data_thi, c_i_data_six, c_l_data, i, overhead)
                        elif action_set[i][0] < action_set[action][0]:
                            Q, N = update_parameters(Q, N, t, c_i_data_thi, c_i_data_six, c_l_data, i, overhead)
                        else:
                            pass
                elif c_i_data_six[t] >= action_set[action][1]:
                    reward = c_i_data_six[t] - c_i_data_thi[t] - overhead[0]
                    predictions.append(df["Six_layer_P"][t])
                    rewards.append(reward)
                    count_2+=1  
                    # Q, N = update_parameters(Q, N, t, c_i_data_thi, c_i_data_six, c_l_data, action, overhead)
                    for i in range(len(action_set)):
                        if action_set[i][0] > action_set[action][0] and action_set[i][1] < action_set[action][1]:
                            Q, N = update_parameters(Q, N, t, c_i_data_thi, c_i_data_six, c_l_data, i, overhead)
                        else:
                            pass
                else:
                    reward = c_l_data[t] - c_i_data_thi[t] - overhead[0] - overhead[1]
                    predictions.append(df["Last_layer"][t])
                    rewards.append(reward)
                    for i in range(len(action_set)):
                        if action_set[i][0] > action_set[action][0] and action_set[i][1] > action_set[action][1]:
                            Q, N = update_parameters(Q, N, t, c_i_data_thi, c_i_data_six, c_l_data, i, overhead)
                        else:
                            pass
                    
                    # Q, N = update_parameters(Q, N, t, c_i_data_thi, c_i_data_six, c_l_data, action, overhead)
                Q, N = update_parameters(Q, N, t, c_i_data_thi, c_i_data_six, c_l_data, action, overhead)
                

        return action_chosen, predictions, rewards, Q, N, count_1, count_2
    
    
    Q = [0 for i in range(len(action_set))]
    N = [0 for i in range(len(action_set))]
    c_tilde = 2
    Q, N = initialize(Q, N, action_set, c_i_data_thi,c_i_data_six, c_l_data, overhead)
    action, preds, rewards, Q, N, count_1, count_2 = one_UCB_cycle(Q, N, c_tilde, c_i_data_thi, c_i_data_six, c_l_data, overhead , df)
    
    optimal_action = Q.index(max(Q))
    print("Optimal_action is ", optimal_action)

    
    
    preds = []
    count = 0
    oracle_reward = []
    for i in range(df.shape[0]):
        
        if c_i_data_thi[i] > action_set[oracle_action][0]:
            reward = 0
            count += 1
            preds.append(df["Thi_layer_P"][i])
            oracle_reward.append(reward)
        elif c_i_data_six[i] > action_set[oracle_action][1]:
            reward = c_i_data_six[i] - c_i_data_thi[i] - overhead[0]
            count += 1
            preds.append(df["Six_layer_P"][i])
            oracle_reward.append(reward)
        else:
            preds.append(df["Last_layer"][i])
            reward = c_l_data[i] - c_i_data_thi[i] - overhead[0] - overhead[1]
            oracle_reward.append(reward)
                
    oracle_reward = np.array(oracle_reward)
    rewards = np.array(rewards)
    regret = (oracle_reward-rewards)
    cumulative_regret = abs(np.cumsum(regret))
    
    
    return cumulative_regret, count_1, count_2, action_set[optimal_action]

In [ ]:
one_cycle(df, action_set, overhead = [0.1, 0.06])

In [14]:
idx = []
for i in range(df.shape[0]):
    if i%50 == 0:
        idx.append(i)

In [15]:
num_cycles = 5
reg_lis = []



for i in range(num_cycles):
    df = df.sample(frac = 1)
    cum_reg,_,_,_ = one_cycle(df, action_set, overhead = [0.06, 0.04])


    reg_lis.append([cum_reg[j] for j in idx])
    
reg_lis = np.array(reg_lis)

KeyError: 'PProb_six'

In [ ]:
len(reg_lis[0])

In [ ]:
lower = []
upper = []

for i in range(len(reg_lis[0])):

    intrval = st.t.interval(alpha=0.95, df=len(reg_lis[:,i])-1,
            loc=np.mean(reg_lis[:,i]),
            scale=st.sem(reg_lis[:,i]))
        
    lower.append(intrval[0])
    upper.append(intrval[1])

In [ ]:
len(lower)

# Other-Exit

In [ ]:
def one_action_cycle(df, action_set, action, overhead):
    c_i_data_thi = []
    for i in df["PProb_thi"]:
        c_i_data_thi.append(i)

    c_i_data_six = []
    for i in df["PProb_six"]:
        c_i_data_six.append(i)

    c_l_data = []
    for i in df["PProb_las"]:
        c_l_data.append(i)

    # number_of_actions = 10
    # action_set = [0.10*((i+10)/2) for i in range(number_of_actions)]
        
    # action_set_1 = [0.10*((i+10)/3) for i in range(20)]
    # action_set = []
    # for i in range(len(action_set_1)):
    #     if i%2 == 0:
    #         action_set.append(action_set_1[i])
    # print(action_set)
    
 
    def oracle_action(overhead):
        oracle_reward = []
        for i in action_set:
            o_reward = 0 
            for t in range(df.shape[0]):
                if c_i_data_thi[t] >= i[0]:
                    reward = 0
    #                 predictions.append(df["Fou_layer_P"][t])
                elif c_i_data_six[t] >= i[1]:
                    reward = c_i_data_six[t] -  c_i_data_thi[t] - overhead[0]
                else:
                    reward = c_l_data[t] - c_i_data_thi[t] - overhead[0] - overhead[1]
                o_reward += reward
    #         print(oracle_reward)
            oracle_reward.append(o_reward/df.shape[0])

        return oracle_reward.index(max(oracle_reward)), oracle_reward
    oracle_action, o_reward = oracle_action(overhead)
    
    
    print("Oracle action =====> ", oracle_action)
    
    
    preds = []
    count = 0
    oracle_reward = []
    for i in range(df.shape[0]):
        if c_i_data_thi[i] > action_set[oracle_action][0]:
            reward = 0
            count += 1
            preds.append(df["Thi_layer_P"][i])
            oracle_reward.append(reward)
        elif c_i_data_six[i] >= action_set[oracle_action][1]:
            reward = c_i_data_six[i] -  c_i_data_thi[i] - overhead[0]
            preds.append(df["Six_layer_P"][i])
            oracle_reward.append(reward)
        else:
            preds.append(df["Last_layer"][i])
            reward = c_l_data[i] - c_i_data_thi[i] - overhead[0] - overhead[1]
            oracle_reward.append(reward)
    
    
#     action = random.choice(action_set)
    print("Action is ", action)
    
#     preds = []
#     count_1 = 0
    rewards = []
    for i in range(df.shape[0]):
        
        if c_i_data_thi[i] >= action[0]:
            reward = 0
#                 count_1 += 1
            preds.append(df["Thi_layer_P"][i])
            rewards.append(reward)
        elif c_i_data_six[i] >= action[1]:
            reward = c_i_data_six[i] - c_i_data_thi[i] - overhead[0]
#                 count_1 += 1
            preds.append(df["Six_layer_P"][i])
            rewards.append(reward)
        else:
            preds.append(df["Last_layer"][i])
            reward = c_l_data[i] - c_i_data_thi[i] - overhead[0] - overhead[1]
            rewards.append(reward)
    
                
    oracle_reward = np.array(oracle_reward)
    rewards = np.array(rewards)
    regret = (oracle_reward-rewards)
    cumulative_regret = abs(np.cumsum(regret))
    # print(oracle_reward,'\n', rewards)
    
    
    return cumulative_regret

In [ ]:
reg_lis_3 = []

for i in range(num_cycles):
    df = df.sample(frac = 1)
    cum_reg_3 = one_action_cycle(df, action_set, [0.5, 0.5], overhead = [0.06, 0.04])  
    reg_lis_3.append([cum_reg_3[j] for j in idx])
    
reg_lis_3 = np.array(reg_lis_3)

In [ ]:
print(reg_lis_3)

In [ ]:
lower_3 = []
upper_3 = []

for i in range(len(reg_lis[0])):
    intrval = st.t.interval(alpha=0.95, df=len(reg_lis_3[:,i])-1,
              loc=np.mean(reg_lis_3[:,i]),
              scale=st.sem(reg_lis_3[:,i]))

        
    lower_3.append(intrval[0])
    upper_3.append(intrval[1])

In [ ]:
len(lower_3)

# $\alpha = 0.8$

In [ ]:
reg_lis_4 = []

for i in range(num_cycles):
    df = df.sample(frac = 1)
    cum_reg_4 = one_action_cycle(df, action_set, [0.8, 0.8], overhead = [0.06, 0.04])

        
    reg_lis_4.append([cum_reg_4[j] for j in idx])
    
reg_lis_4 = np.array(reg_lis_4)

In [ ]:
print(reg_lis_4)

In [ ]:
upper_4 = []
lower_4 = []

for i in range(len(reg_lis[0])):
    intrval = st.t.interval(alpha=0.95, df=len(reg_lis_4[:,i])-1,
              loc=np.mean(reg_lis_4[:,i]),
              scale=st.sem(reg_lis_4[:,i]))
        
    lower_4.append(intrval[0])
    upper_4.append(intrval[1])

# $\alpha = 0.9$  

In [ ]:
reg_lis_5 = []
for i in range(num_cycles):
    df = df.sample(frac = 1)
    cum_reg_5 = one_action_cycle(df, action_set, [0.9, 0.8], overhead = [0.06, 0.04])
        
    reg_lis_5.append([cum_reg_5[j] for j in idx])
    
reg_lis_5 = np.array(reg_lis_5)

In [ ]:
print(reg_lis_5)

In [ ]:
lower_5 = []
upper_5 = []

for i in range(len(reg_lis[0])):
    intrval = st.t.interval(alpha=0.95, df=len(reg_lis_5[:,i])-1,
              loc=np.mean(reg_lis_5[:,i]),
              scale=st.sem(reg_lis_5[:,i]))
        
    lower_5.append(intrval[0])
    upper_5.append(intrval[1])

In [ ]:
len(lower_5)

# $Final-Exit$ 

In [ ]:
reg_lis_fin = []
for i in range(num_cycles):
    df = df.sample(frac = 1)
    cum_reg_fin = one_action_cycle(df, action_set, [1.0, 0.9], overhead = [0.06, 0.04])
        
    reg_lis_fin.append([cum_reg_fin[j] for j in idx])
    
reg_lis_fin = np.array(reg_lis_fin)

In [ ]:
print(reg_lis_fin)

In [ ]:
lower_fin = []
upper_fin = []

for i in range(len(reg_lis[0])):
    intrval = st.t.interval(alpha=0.95, df=len(reg_lis_fin[:,i])-1,
              loc=np.mean(reg_lis_fin[:,i]),
              scale=st.sem(reg_lis_fin[:,i]))
        
    lower_fin.append(intrval[0])
    upper_fin.append(intrval[1])

In [ ]:
x = [i for i in range(len(idx))]
plt.plot(reg_lis.mean(axis = 0) ,'g', label="UBERT",linestyle = 'solid')
plt.fill_between(x, lower, upper, color='green', alpha=0.1)
plt.plot(reg_lis_3.mean(axis = 0) ,'orange', label="alpha = [0.6, 0.5]", linestyle = 'dashed')
plt.fill_between(x, lower_3, upper_3, color='orange', alpha=0.1)
plt.plot(reg_lis_4.mean(axis = 0) ,'red', label="alpha = [0.9, 0.8]", linestyle = 'dashdot')
plt.fill_between(x, lower_4, upper_4, color='red', alpha=0.1)
plt.plot(reg_lis_5.mean(axis = 0) ,'blue', label="alpha = [0.5,0.5]",linestyle = 'dotted')
plt.fill_between(x, lower_5, upper_5, color='blue', alpha=0.1)
plt.plot(reg_lis_fin.mean(axis = 0) ,'brown', label="alpha = [1.0, 0.9](Final-Exit)",linestyle = 'dashed')
plt.fill_between(x, lower_fin, upper_fin, color='brown', alpha=0.1)
plt.legend()
plt.xlabel("Time horizon")
plt.ylabel("Cumulative regret")
plt.title("IMDb")
plt.savefig('/home/divya/UBERT/Regret_plot_reduced/Regret_plot_imdb.pdf')
plt.show()

In [ ]:
# num_cycles = 5
def accuracy_lat_calcu(df, optimal_action, overhead = [0.06, 0.04]):
    c_i_data_thi = []
    for i in df["PProb_thi"]:
        c_i_data_thi.append(i)

    c_i_data_six = []
    for i in df["PProb_six"]:
        c_i_data_six.append(i)

    c_l_data = []
    for i in df["PProb_las"]:
        c_l_data.append(i)

    preds = []
    count_1 = 0
    count_2 = 0
    for i in range(df.shape[0]):
        
        if c_i_data_thi[i] >= optimal_action[0]:
            # reward = 0
            count_1+=1
#                 count_1 += 1
            preds.append(df["Thi_layer_P"][i])
        elif c_i_data_six[i] >= optimal_action[1]:
            # reward = c_i_data_six[i] - c_i_data_thi[i] - overhead[0]
            count_2+=1  
            preds.append(df["Six_layer_P"][i])
            # rewards.append(reward)
        else:
            preds.append(df["Last_layer"][i])
            # reward = c_l_data[i] - c_i_data_thi[i] - overhead[0] - overhead[1]
            # rewards.append(reward)
    accuracy = 0
    for i in range(df.shape[0]):
        if preds[i] == df["True_labels"][i]:
            accuracy+=1
        else:
            pass
    return accuracy/df.shape[0], count_1/df.shape[0], count_2/df.shape[0]

    

In [ ]:
num_cycles = 5
accuracy_lis = []
latency_lis = []
for i in range(num_cycles):
    df = df.sample(frac = 1)
    _,_,_,optimal_action = one_cycle(df, action_set, overhead = [0.06, 0.04])
    accuracy, count_1, count_2 = accuracy_lat_calcu(df, optimal_action, overhead = [0.06, 0.04])
    latency = count_1+count_2
    print("Optimal action is:", optimal_action,
          "\n=============================\n Fraction of Exits at first layer",count_1,
          "\n=============================\n Fraction of Exits at second layer", count_2,
          "\n=============================\n Total latency", 1-(count_1+count_2)
              )
    accuracy_lis.append(accuracy)
    latency_lis.append(latency)
accuracy_lis = np.array(accuracy_lis)
latency_lis = np.array(latency_lis)
    

In [ ]:
print(np.mean(accuracy_lis))

In [ ]:
print(np.mean(latency_lis))